## seasonal patterns of NLCT events

* 2023 Dec 22: fixed to only ingest NCLT data in netcdf form (previously would use picked xarray data)

In [ ]:
import glob
import xarray as xr
import pickle
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
region = "georges_bank"
nlct_path = "/mnt/data2/jturner/nlct_data/"+region+"/"

Creating seasonal average for month: 

In [ ]:
month = '01' #mm
month_name = 'Jan'

In [ ]:
files = sorted(glob.glob(nlct_path+'202?'+month+'*/*06H_00M.nc'))

In [ ]:
for file in files: 
    print(file.split('/')[-2])

count = len(files)
print(count)

Each NLCT file is a saved xarray DataArray, they need the dictionary to be appended into a Dataset:

In [ ]:
datasets = []

for file in files:
    ds = xr.open_dataset(file)
    ds['__xarray_dataarray_variable__'] = xr.where(ds['__xarray_dataarray_variable__'] < 0, 0, ds['__xarray_dataarray_variable__'])
    datasets.append(ds)
    
nlct_ds = xr.concat(datasets, dim='time')

In [ ]:
#--- Georges Bank, Falkland no cropping
max_x = np.max(nlct_ds.x.values)
min_x = np.min(nlct_ds.x.values)
max_y = np.max(nlct_ds.y.values)
min_y = np.min(nlct_ds.y.values)

#--- Oaxaca cropping adjustments
# max_x = np.max(nlct_ds.x.values) -0.025 
# min_x = np.min(nlct_ds.x.values) +0.025
# max_y = np.max(nlct_ds.y.values) -0.005
# min_y = np.min(nlct_ds.y.values) +0.005

x_range = slice(min_x, max_x)
y_range = slice(max_y, min_y)

nlct_ds_sm = nlct_ds.sel(x=x_range, y=y_range)

In [ ]:
btd_tot = nlct_ds_sm['__xarray_dataarray_variable__'].sum(dim='time')
btd_mean = btd_tot/count

In [ ]:
overall_mean = btd_mean.mean(dim='y')
overall_mean = overall_mean.mean(dim='x')

In [ ]:
high_lim = overall_mean.values
low_lim = overall_mean.values

In [ ]:
projection=ccrs.PlateCarree()
fig,ax=plt.subplots(1, figsize=(12,12),subplot_kw={'projection': projection})
cmap = plt.cm.Spectral_r
levels = np.linspace(0, 1, 21)

#--- Falklands needs set extent
# min_lon = -75
# min_lat = -58
# max_lon = -50
# max_lat = -35
# ax.set_extent([min_lon, max_lon, min_lat, max_lat])

c=ax.contourf(btd_mean.lon, btd_mean.lat, btd_mean, cmap=cmap, extend='both', levels=levels)
clb=plt.colorbar(c, shrink=0.3, pad=0.02, ax=ax)
ax.set_title('Average '+month_name+' NLCT Positive BTD (2022-2023)', fontsize=24)
clb.set_label('BTD (K)')

ax.add_feature(cfeature.STATES)
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.COASTLINE)

#plt.savefig('NLCT_Seasonal/'+month_name+'_nclt.png', bbox_inches='tight')

### Running climatology for multiple months

In [ ]:
def Calc_BTD_mean(month_id):
    
    #--- Create month number
    month_num = str(month_id+1).zfill(2)
    
    print(month_num)
    files = sorted(glob.glob(nlct_path+'202?'+month_num+'*/*06H_00M.nc'))
    count = len(files)
    datasets = []

    for file in files:
        ds = xr.open_dataset(file)
        #--- Set negative values to zero
        ds['__xarray_dataarray_variable__'] = xr.where(ds['__xarray_dataarray_variable__'] < 0, 0, ds['__xarray_dataarray_variable__'])
        datasets.append(ds)

    nlct_ds = xr.concat(datasets, dim='time')
    nlct_ds_sm = nlct_ds.sel(x=x_range, y=y_range)
    btd_tot = nlct_ds_sm['__xarray_dataarray_variable__'].sum(dim='time')
    btd_mean = btd_tot/count
    
    return btd_mean

Throws an error if there is not data for each month

In [ ]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

#--- Calculate the number of rows and columns for the subplots
num_months = len(months)
num_rows = int(num_months**0.5)
num_cols = (num_months + num_rows - 1) // num_rows

#--- Create a figure and an array of subplots
projection=ccrs.PlateCarree()
fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 8), subplot_kw={'projection': projection})
cmap = plt.cm.Spectral_r
levels = np.linspace(0, 1, 21)

#--- Flatten the 2D array of subplots to simplify indexing
axes = axes.flatten()

#--- Falklands needs set extent
min_lon = -75
min_lat = -58
max_lon = -50
max_lat = -35

for i, month in enumerate(months):
    #--- Falklands needs set extent
    #axes[i].set_extent([min_lon, max_lon, min_lat, max_lat])
    
    btd_mean = Calc_BTD_mean(i)
    c=axes[i].contourf(btd_mean.lon, btd_mean.lat, btd_mean, cmap=cmap, extend='both', levels=levels)
    axes[i].set_title(month, fontsize=14)
    axes[i].add_feature(cfeature.STATES)
    axes[i].add_feature(cfeature.BORDERS)
    axes[i].add_feature(cfeature.COASTLINE)

    
cbar = plt.colorbar(c, ax=axes, orientation='vertical', fraction=0.02, pad=0.1)
cbar.set_label('BTD (K)')  

    
fig.suptitle('Composite NLCT Positive BTD (2022-2023)', fontsize=24)
fig.set_dpi(200)
plt.savefig('nclt_seasonal_'+region+'.png', dpi=200, bbox_inches='tight')